In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from keras.utils import np_utils

plt.style.use("fivethirtyeight")
%matplotlib inline

In [ ]:
# fetch data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

n_dimensions = mnist.train.images[0].shape[0]
n_classes = mnist.train.labels[0].shape[0]

n_epochs = 1000
batch_size = 32
lr = 1e-2

In [ ]:
# construct graph

tf.reset_default_graph()

with tf.variable_scope("inputs"):
    X_ = tf.placeholder(tf.float32, [None, n_dimensions], name="X")
    y_ = tf.placeholder(tf.float32, [None, n_classes], name="y")

with tf.variable_scope("model"):
    fc1 = tf.layers.dense(inputs=X_, units=n_classes, name='fc1', activation=tf.nn.relu)  
    
with tf.variable_scope('model/fc1', reuse=True):
    w = tf.get_variable(name='kernel')

probs = tf.nn.softmax(fc1)
loss = tf.losses.log_loss(labels=y_, predictions=probs)

with tf.variable_scope("step"):
    grad = tf.gradients(loss, w)[0]  # raw gradients of loss function wrt params
    update_param = w.assign(w - lr * grad)
    
init = tf.global_variables_initializer()

In [ ]:
ls_loss = []

with tf.Session() as sess:  
    sess.run(init)
    
    for epoch in range(n_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        # first update parameter, then compute loss
        _, loss_ = sess.run([update_param, loss], feed_dict={X_: batch_xs, y_: batch_ys})
        ls_loss.append(loss_)
        
# pretty plot
rolling_plot = pd.Series(ls_sgd_loss_).rolling(window=100).mean()
plt.plot(rolling_plot)
plt.xlabel("epochs")
plt.ylabel("loss")  # linear decrease since gradients do not accumulate
plt.title("sgd loss")